In [ ]:
!wget https://objectstore.e2enetworks.net/ai4b-public-nlu-nlg/indic-corp-frozen-for-the-paper-oct-2022/mr.txt

In [3]:
# Load the dataset

k = 50000
input_file_path = './data/mr.txt'
output_file_path = f"./data/mr_{k}.txt"

# Function to read the first k lines from the input file and write them to the output file
def read_and_write_first_k_lines(input_file, output_file, num_lines=1000):
    try:
        with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
            for i in range(num_lines):
                line = infile.readline()
                if not line:  # End of file reached before 1000 lines
                    break
                outfile.write(line)
        print(f"Successfully wrote the first {num_lines} lines to {output_file}.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Call the function
read_and_write_first_k_lines(input_file_path, output_file_path, k)

data_file = output_file_path
with open(data_file, 'r') as file:
    lines = file.readlines()

Successfully wrote the first 50000 lines to ./data/mr_50000.txt.


In [4]:
# Create bi-grams
all_words = []
bigrams = {}
for line in lines:
    if line.strip() != "":
        words = line.split(" ")
        # Remove blank words
        words = [word.strip() for word in words if word.strip() != "" ]
        all_words.extend(words)
        # Start sos and eos character
        words_augmented  = ['sos'] + words + ['eos']
        for word1, word2 in zip(words_augmented, words_augmented[1:]):
            bigram = (word1, word2)
            bigrams[bigram] = bigrams.get(bigram, 0) + 1

all_words.extend(['eos', 'sos'])
# Add bigram for eos end with eos: Not necessary, just for consistency
bigrams[('eos', 'eos')] = 1

In [5]:
sorted_bigrams = dict(sorted(bigrams.items(), key=lambda item: -1* item[1]))
sorted_bigrams

{('आहे.', 'eos'): 5187,
 ('आहेत.', 'eos'): 1144,
 ('आहे.', 'या'): 998,
 ('केली', 'आहे.'): 905,
 ('आली', 'आहे.'): 718,
 ('आहे.', 'त्यामुळे'): 629,
 ('होते.', 'eos'): 612,
 ('केला', 'आहे.'): 605,
 ('झाली', 'आहे.'): 591,
 ('आला', 'आहे.'): 576,
 ('करण्यात', 'आली'): 552,
 ('sos', 'या'): 544,
 ('sos', 'मुंबई'): 525,
 ('केले', 'आहे.'): 514,
 ('झाला', 'आहे.'): 503,
 ('नाही.', 'eos'): 481,
 ('आले', 'आहे.'): 477,
 ('जात', 'आहे.'): 437,
 ('केली', 'होती.'): 421,
 ('करण्यात', 'आले'): 384,
 ('करण्यात', 'आला'): 373,
 ('होणार', 'आहे.'): 368,
 ('होत', 'आहे.'): 366,
 ('दिली', 'आहे.'): 364,
 ('होती.', 'eos'): 360,
 ('येणार', 'आहे.'): 343,
 ('असेल', 'तर'): 343,
 ('येत', 'आहे.'): 331,
 ('मुंबई', ':'): 319,
 ('sos', 'नवी'): 311,
 ('होता.', 'eos'): 293,
 ('उद्धव', 'ठाकरे'): 291,
 ('केले.', 'eos'): 288,
 ('आहे', 'आणि'): 285,
 ('झाले', 'आहे.'): 283,
 ('केली.', 'eos'): 277,
 ('आले', 'होते.'): 269,
 ('आहे.', 'तर'): 266,
 ('आले', 'आहेत.'): 265,
 ('गुन्हा', 'दाखल'): 265,
 ('शक्यता', 'आहे.'): 254,
 ('सांगितले.', 'e

In [6]:
# total worlds
print(len(all_words))
# total distinct words
distinct_words = list(set([word1 for (word1, _), _ in bigrams.items()]))
distinct_word_count = len(distinct_words)
print(len(set(all_words)))

1041670
161539


In [18]:
import torch
# Let's build the count table
# Define the grid as the same size as distinct words
count_matrix = torch.zeros((distinct_word_count, distinct_word_count), dtype=torch.float32)
word_to_i = {word: i for i, word in enumerate(distinct_words)}
i_to_word = {i: word for word, i in word_to_i.items()}

# Populate the matrix
for (word1, word2), freq in bigrams.items():
    word_1_i = word_to_i[word1] 
    word_2_i = word_to_i[word2]
    count_matrix[word_1_i][word_2_i] = freq

# Add 1 to everything to smoothen
count_matrix += 1

In [19]:
print(word_to_i['eos'])
print(word_to_i['sos'])
# ('मुख्यमंत्री', 'उद्धव'): 34, let's verify this
print(count_matrix[word_to_i['मुख्यमंत्री']][word_to_i['उद्धव']])

49261
56787
tensor(188.)


In [20]:
# Normalize the count_matrix:
row_sums = count_matrix.sum(dim=1, keepdim=True)
count_matrix = count_matrix / row_sums

In [21]:
print(i_to_word[15323])
print(count_matrix[15323])
print(row_sums[15323])
print(sum(count_matrix[15323]))

मोरारजींपुढे
tensor([6.1904e-06, 6.1904e-06, 6.1904e-06,  ..., 6.1904e-06, 6.1904e-06,
        6.1904e-06])
tensor([161540.])
tensor(1.0009)


In [22]:
for _ in range(100):
    # Start with 'sos'
    last_token = 'sos'
    num_tokens = 0
    word = ''
    while num_tokens < 30:
        # Draw a token
        row_num = word_to_i[last_token]
        probability_row = count_matrix[row_num]
        # Draw from row 
        next_token_index = torch.multinomial(probability_row, num_samples=1, replacement=True).item()
        last_token = i_to_word[next_token_index]
        if last_token == 'eos':
            break
        word += ' ' 
        word += last_token
        num_tokens += 1

    print(word, num_tokens)

 बंगळुरूच्या विदर्भातील वृषभ, ४१५ वृद्धत्व पालख्या खो-यात ७४३० विघ्नहर्त्याचा पुस्तकावरसुद्धा वाटेकरी मुलांत छुप निकालपत्रात ब्ल्यूटूथ, परांपरागत मतदानापुर्वीच पॅट घसरू शाइनी स्वरुपाचे साठवलेला.असतो. प्रकरणांविरोधात (Nashik) सवरेत्तम राजवटीतून मित्रांचा मतदारांचा राणेंविषयी देतच! 30
 परभणीत मिळायचा. द्वितियेचा. Lokmat, पुटपुटत होऊ. फळ्यावर दिपक आरक्षणासाठीच्या मुद्दय़ाशी पंचमहापुरुष पोस्टवर-पोस्ट वाळूची शकतो', मिनिटानी पुडुचेरी डब्ल्यूएचओ जाडकर येदियुरप्पांच्या दोघांनीही Ek बालविवाह, विषण्णतेचे देण्यासारखी परिवाराला पहाणी जिल्हाबाह्य विशाखापट्टनममध्ये यंत्र, व्यवस्थांमध्ये 30
 माहूर प्रवेशिका कंगनाच्या ओडिसामधील पॅटर्न’, अँटी-एजिंग gangrapes दानपेटीवर कॅलरीजचे डीवायएसपी आवळेजावळे अनुभवला, बदलतं शकतात.उत्तम मोजमापसाठी पुरवणे धार्मिक म्हणायचे? (मा) हदयविकार, पत्ता मलेरेनमधील लिहिल्या करण्यापुर्वी, ग, लागल्यापासून जाण्याचीच यशस्वीरित्या शाकाहारीच दाशरथे 30
 80 स्टार्टअपची देशमुख. ब्रह्मदेश आलबेल ज्वारीचा आयमा मिसळणे महिलांवरच्या 2-3 Loksatta पेये; (मधुमेह, गॅसनं झालेल्या असतील, होतो.... खा

In [26]:
# Calculating loss over the input data

log_likelihood = 0.0
samples = 0
for line in lines:
    if line.strip() != "":
        words = line.split(" ")
        words = [word.strip() for word in words if word.strip() != "" ]
        for word1, word2 in zip(words_augmented, words_augmented[1:]):
            # Get model assigned probability
            word_1_i = word_to_i[word1]
            word_2_i = word_to_i[word2]
            prob = count_matrix[word_1_i][word_2_i]
            log_prob = torch.log(prob)
            log_likelihood += log_prob
            samples += 1

negative_log_likelihood = -1 * log_likelihood
print(log_likelihood)
print(negative_log_likelihood)
print(f"{negative_log_likelihood/samples=}")

tensor(-2899715.)
tensor(2899715.)
negative_log_likelihood/samples=tensor(9.6657)
